## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

5. Add your OpenAI key.  OpenAI is only needed for example purposes.  If you have your own model, you can substitute those in it's place.

In [ ]:
%pip install okareo
%pip install openai

In [2]:
# Setup the environment for your notebook
import os

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [3]:
# Simple summarization prompt using OpenAI'a GPT 3.5 Turbo model

from openai import OpenAI
import json

client = OpenAI(api_key=OPENAI_API_KEY)

def get_turbo_summary(messages, model="gpt-3.5-turbo", 
  temperature=0, max_tokens=500):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, 
    max_tokens=max_tokens,
  )
  return response.choices[0].message.content

USER_PROMPT_TEMPLATE = "{input}"

SUMMARIZATION_CONTEXT_TEMPLATE = """
You will be provided with text.
Summarize the text in 1-3 simple sentences.
If the text's title and the author's name are available, refer to both in the summary.
Your goal is to make the summary clear to a middle school student.
"""


In [4]:
# Text for the summarization model to process
text_1 = """
I Wandered Lonely as a Cloud

I wandered lonely as a cloud
That floats on high o'er vales and hills,
When all at once I saw a crowd,
A host, of golden daffodils;
Beside the lake, beneath the trees,
Fluttering and dancing in the breeze.

Continuous as the stars that shine
And twinkle on the milky way,
They stretched in never-ending line
Along the margin of a bay:
Ten thousand saw I at a glance,
Tossing their heads in sprightly dance.

The waves beside them danced; but they
Out-did the sparkling waves in glee:
A poet could not but be gay,
In such a jocund company:
I gazed-and gazed-but little thought
What wealth the show to me had brought:

For oft, when on my couch I lie
In vacant or in pensive mood,
They flash upon that inward eye
Which is the bliss of solitude;
And then my heart with pleasure fills,
And dances with the daffodils.

-- by William Wordsworth
"""

text_2 = """
All Things Are Current Found

ALL things are current found
On earthly ground,
Spirits and elements
Have their descents.

Night and day, year on year,
High and low, far and near,
These are our own aspects,
These are our own regrets.

Ye gods of the shore,
Who abide evermore,
I see you far headland,
Stretching on either hand;

I hear the sweet evening sounds
From your undecaying grounds;
Cheat me no more with time,
Take me to your clime.

-- by Henry D. Thoreau
"""

text_3 = """
Hope Is the Thing with Feathers

"Hope" is the thing with feathers-
That perches in the soul-
And sings the tune without the words-
And never stops-at all-

And sweetest-in the Gale-is heard-
And sore must be the storm-
That could abash the little Bird
That kept so many warm-

I've heard it in the chillest land-
And on the strangest Sea-
Yet, never, in Extremity,
It asked a crumb-of Me.

-- by Emily Dickinson
"""

In [5]:
# Example gpt call summarizing text (a poem :) 

messages =  [  
    {'role':'system', 'content': SUMMARIZATION_CONTEXT_TEMPLATE},    
    {'role':'user', 'content': text_3}
] 

response = get_turbo_summary(messages)
print(response)

The poem "Hope Is the Thing with Feathers" by Emily Dickinson describes hope as a bird that lives in our souls and sings a beautiful song. It never goes away, even in the toughest times, and doesn't ask for anything in return.


In [8]:
# Create a scenario to evaluate the model with
import uuid
import random
import string
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData, ScenarioType

random_string = ''.join(random.choices(string.ascii_letters, k=5))
okareo = Okareo(OKAREO_API_KEY)

scenario_set_create = ScenarioSetCreate(
    name=f"Summarization - {random_string}",
    number_examples=1,
    generation_type=ScenarioType.SEED,
    seed_data=[
        SeedData(
            input_=text_1,  
            result=str(uuid.uuid4())
        ),
        SeedData(
            input_=text_2,  
            result=str(uuid.uuid4())
        ),
        SeedData(
            input_=text_3, 
            result=str(uuid.uuid4())
        )
    ],
)
scenario = okareo.create_scenario_set(scenario_set_create)


SCENARIO_ID = scenario.scenario_id
PROJECT_ID = scenario.project_id
print(scenario)
print(f"https://app.okareo.com/project/{PROJECT_ID}/scenario/{SCENARIO_ID}")

ScenarioSetResponse(scenario_id='0d861414-22b4-4a43-a614-733df0ba2e1b', project_id='394c2c12-be7a-47a6-911b-d6c673bc543b', time_created=datetime.datetime(2024, 1, 18, 22, 3, 2, 597323), type='SEED', tags=None, name='Summarization - NCnNA', seed_data=[SeedData(input_="\nI Wandered Lonely as a Cloud\n\nI wandered lonely as a cloud\nThat floats on high o'er vales and hills,\nWhen all at once I saw a crowd,\nA host, of golden daffodils;\nBeside the lake, beneath the trees,\nFluttering and dancing in the breeze.\n\nContinuous as the stars that shine\nAnd twinkle on the milky way,\nThey stretched in never-ending line\nAlong the margin of a bay:\nTen thousand saw I at a glance,\nTossing their heads in sprightly dance.\n\nThe waves beside them danced; but they\nOut-did the sparkling waves in glee:\nA poet could not but be gay,\nIn such a jocund company:\nI gazed-and gazed-but little thought\nWhat wealth the show to me had brought:\n\nFor oft, when on my couch I lie\nIn vacant or in pensive moo

In [9]:
# Evaluate the scenario and model combination and then get a link to the results on Okareo

import random
import string
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData
from okareo.model_under_test import OpenAIModel
from okareo_api_client.models.test_run_type import TestRunType

okareo = Okareo(OKAREO_API_KEY)
random_string = ''.join(random.choices(string.ascii_letters, k=5))
mut_name = f"OpenAI Summarization Model - {random_string}"
eval_name = f"Summarization Run - {random_string}"

model_under_test = okareo.register_model(
    name=mut_name,
    model=OpenAIModel(
        model_id="gpt-3.5-turbo",
        temperature=0,
        system_prompt_template=SUMMARIZATION_CONTEXT_TEMPLATE,
        user_prompt_template=USER_PROMPT_TEMPLATE,
    ),
)

evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=scenario,
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION,
    calculate_metrics=True,
)


EVAL_ID = evaluation.id
PROJECT_ID = evaluation.project_id
print(f"https://app.okareo.com/project/{PROJECT_ID}/eval/{EVAL_ID}")

https://app.okareo.com/project/394c2c12-be7a-47a6-911b-d6c673bc543b/eval/feed244f-e1b5-46d0-8e34-078df0c27667
